# Elastic tensors

For the source code, see [elastic](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.elastic.elastic_workgraph).

## Visualizing the WorkGraph Builder


In [ ]:
from workgraph_collections.ase.espresso.elastic import elastic_workgraph

node = elastic_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [ ]:
from workgraph_collections.ase.espresso.elastic import elastic_workgraph
from aiida import load_profile

load_profile()

wg = elastic_workgraph()
wg.to_html()

## Example: Silicon

### Prepare the inputs and submit the workflow


In [15]:
from ase.build import bulk
from aiida import load_profile
from workgraph_collections.ase.espresso.elastic import elastic_workgraph

load_profile()

# create input structure node
atoms = bulk("Al")

metadata = {
    "options": {
        'prepend_text' : """
        module load cray/21.12  cpeIntel/21.12 QuantumESPRESSO/7.2
        eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}

metadata_eiger = {
    "options": {
        "custom_scheduler_commands": "#SBATCH --account=mr32",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 36,
        },
    }
}

pseudopotentials = {"Al": "Al.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "control": {"calculation": "relax", 
                "tstress": True,
                "tprnfor": True,
                "verbosity": "high",
                "etot_conv_thr": 1e-7,
                "forc_conv_thr": 1.0e-6,
                },
    "system": {"ecutwfc": 40, "ecutrho": 320,
               "occupations": "smearing",
               "smearing": "gaussian",
                "degauss": 0.01,
                },
    "electrons": {"conv_thr": 1e-11}
}
#------------------------- Set the inputs -------------------------
wg = elastic_workgraph(atoms=atoms,
                   computer="localhost",
                   command="mpirun -np 2 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=(24, 24, 24),
                   metadata=metadata)
#------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=300)


WorkGraph node created, PK: 62671


<WorkChainNode: uuid: b05ac0ef-aeda-4a42-8e50-18ad9621215c (pk: 62671) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

In [13]:
#------------------------- Print the output -------------------------
# 1 eV/Angstrom3 = 160.21766208 GPa
import numpy as np
print("Result: ")
elastic_constants = wg.nodes["fit_elastic"].outputs["result"].value.value
print(np.round(elastic_constants, 2))


Result: 
[[ 0.82  0.32  0.32 -0.   -0.03  0.03]
 [ 0.32  0.82  0.32  0.03 -0.   -0.03]
 [ 0.32  0.32  0.82 -0.03  0.03  0.  ]
 [ 0.    0.   -0.    0.24 -0.   -0.  ]
 [ 0.   -0.    0.    0.    0.24  0.  ]
 [-0.   -0.    0.    0.   -0.    0.24]]
